# Qu'allons nous faire dans ce TP ? 🏗

Ce TP est là pour vous montrer que des problématiques GL peuvent être en partie solutionnées par des solutions d'IA. Ce deuxième TP va vous montrer l'utilisation de méthodes de traitement du langage naturel (*Natural Language Processing (NLP)*) pour de la classification de tickets et plus particulièrement de tickets de bugs. Cet exemple est issu de nos travaux de recherche, la référence du papier de recherche est ici : 

*   *Quentin Perez, Christelle Urtado, Sylvain Vauttier. Bug or not bug? That is the question. ICPC 2021 - 29th IEEE/ACM International Conference on Program Comprehension, May 2021, Online, France. pp.443-452*
*   URL vers le papier: [https://hal.mines-ales.fr/hal-03177423/document](https://hal.mines-ales.fr/hal-03177423/document)


Cependant attention, l'IA 🤖 n'est pas non plus Merlin l'Enchanteur 🧙 et ne peut donc pas aller au-delà de ses capacités. Elle est fortement dépendante du type d'apprentissage, de la méthode d'apprentissage et des données d'entrées. 

# Prérequis pour l'utilisation du notebook Colab **📦**

## ⚠ AVANT TOUTES MANIPULATIONS : FAIRE COPIE DE CE NOTEBOOK COLAB DANS VOTRE ESPACE GOOGLE

Pour ce faire aller sur le menu "Fichier" puis "Enregistrer une copie dans Drive"

## Installation des bibliothèques et téléchargement des fichiers

Import du dataset de tickets

In [ ]:
!wget https://raw.githubusercontent.com/qperez/TP-Master-MTP-GL-IA4GL/main/dataset_herzig_etal.json

--2022-11-28 11:17:45--  https://raw.githubusercontent.com/qperez/TP-Master-MTP-GL-IA4GL/main/dataset_herzig_etal.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5326382 (5.1M) [text/plain]
Saving to: ‘dataset_herzig_etal.json.4’

dataset_herzig_etal 100%[===================>]   5.08M  --.-KB/s    in 0.02s   

2022-11-28 11:17:45 (279 MB/s) - ‘dataset_herzig_etal.json.4’ saved [5326382/5326382]



Installation de librairie permettant l'explicabilité du classifieur

In [ ]:
!pip install eli5==0.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import des lib nécessaires au Notebook.

In [ ]:
import codecs
import json
import time

import matplotlib.pyplot as plt
import numpy as np
from eli5.lime import TextExplainer
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC

## Fonctions nécessaires au TP

### Chargement du dataset

Fonction permettant l'ouverture d'un jeu de ticket sauvegardé au format json.

In [ ]:
def load_dataset(json_path):
    raw_data = []
    with codecs.open(json_path, "r", "utf-8") as fin:
        raw_data += json.load(fin)
    return raw_data

### Fonction de multiplication des titres des tickets

Multiplication du titre des tickets par le facteur donné en paramètre. 

In [ ]:
def multiply_title(dataset, factor):
    for ticket in dataset:
        for i in range(1,factor):
            ticket["title"] += " " + ticket["title"]

### Fonction de binarisation des labels

Permet de convertir les labels "NBUG" et "BUG" en une information binaire 0 ou 1
* 0 = NBUG
* 1 = BUG

In [ ]:
def binarization_labels(labels):
    return np.ravel(label_binarize(labels, classes=["NBUG","BUG"]))

### Fonction de récupération du corpus et des étiquettes

Cette fonction retourne deux tableaux :
* Le premier est un tableau contenant les informations textuelles des tickets (pour chaque ticket, le title est concaténé au body)
* Le second tableau contient les labels textuelles associées à chacun des tickets.

In [ ]:
def get_corpus_labels(raw_data):
# Corpus building.
    corpus = []
    labels = []
    n_bug = 0
    for n_file in raw_data:
        corpus.append(n_file["title"] + " " + n_file["body"])
        labels.append(n_file["label"])
        if n_file["label"] == "BUG":
            n_bug += 1
    print(f"{n_bug} BUG / {len(labels)} \n")
    return corpus, labels

### Fonction d'évaluation d'un classifieur avec un train/test split

Cette fonction permet l'évaluation d'un classifieur sur la base d'une division du jeu de données en 2 parties :
* une partie dédiée à l'entraînement (partie dite _train_)
* une partie dédiée au test du classifieur (partie dite _test_)
Ici 77% des données du dataset sont destinées à l'entraînement et 33% au test du classifieur. 
La fonction calcule les mesures F1, rappel et précision. Elle dispose de 4 paramètres :
* X : le corpus complet vectorisé
* binarized_labels : la liste des labels binarisés
* clf : le classifieur à évaluer

In [ ]:
def make_scoring_train_test_split(X,binarized_labels, clf, test_size=33):
    X_train, X_test, y_train, y_test = train_test_split(X, binarized_labels, test_size=test_size, random_state=42)

    start_time = time.time()
    #scores = cross_val_score(clf, X, binarized_labels, cv=cv, scoring='f1')
    print("--- Start training ---",flush=True)
    clf.fit(X_train, y_train)
    print("--- %s seconds for training ---" % (time.time() - start_time),flush=True)
    y_pred = clf.predict(X_test)

    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)

    print("F1 score train/test: %0.3f" % f1)
    print("Recall score train/test: %0.3f" % recall)
    print("Precision score train/test: %0.3f" % precision)
    print()
    return f1



---



# Début du TP 🚀

### 📖 ➡ 🤖 Fonction de vectorisation et de calcul des features les plus représentatives
Cette fonction prend 4 paramètres en entrée : 
* corpus : tableau d'éléments textuels extraits avec la fonction <code>get_corpus_labels</code>
* labels : tableau de labels binarisés avec la fonction <code>binarization_labels</code>
* vectorizer : objet vectorizer pour transformer le corpus 
* k_best : nombre de features représentatives du corpus à sélectionner à l'aide du chi-deux
* print_feature_names : afficher ou non les features sélectionnées par le chi2


Vous allez devoir compléter la fonction <code>feature_computing</code>, pour cela :
1. Utilisez le vectorizer passé en paramètre afin de transformer les informations textuelles en informations mathématiques (vecteurs) utilisables par des classifieurs. Ces vecteurs sont créés à l'aide de la méthode _Term Frequency-Inverse Document Frequency_ ([wikipédia TF-IDF](https://fr.wikipedia.org/wiki/TF-IDF)) Pour cela référez vous à la fonction <code>fit_transform</code> ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html?highlight=tfidf#sklearn.feature_extraction.text.TfidfVectorizer.fit_transform)). Vous affecterez le retour de <code>fit_transform</code> à une variable nommée <code>X</code>. 
2. La seconde étape consiste à sélectionner les features les plus représentatives du corpus. Un moyen de faire cela est d'utiliser la méthode du khi-deux. La méthode du khi-deux va mesurer la dépendance entre une feature donnée et la classe (BUG ou NBUG) et ainsi vous permettre de sélectionner <code>k</code> features représentatives du corpus. Pour faire cela vous devez créer un objet <code>SelectKBest</code> que vous affecterez à la variable <code>ch2</code>. Vous pouvez vous inspirer de l'exemple donné dans la [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=selectkbest%20fit_transform).
3. Utiliser la méthode <code>fit_transform</code> avec en paramètres <code>X</code> et les <code>labels</code>. Affectez le retour de cette méthode à la variable <code>X</code> 

Si vous souhaitez visualiser les features sélectionnées par le chi2 vous pouvez passer la variable <code>print_feature_names</code> à <code>True</code>

In [ ]:
def feature_computing(corpus, binarized_labels, vectorizer, k_best=30000, print_feature_names=True):
    # TF-IDF.
    start_time = time.time()
    print("--- Start feature computing ---")
    #Placer ici la ligne permettant de vectoriser le corpus avec fit_transform
    X = vectorizer.fit_transform(corpus,None)


    print(f"\t{X.shape[1]} features.")

    print("Extracting %d best features by a chi-squared test" % k_best)
    #Placer ici le code pour extraire les features
    ch2 = SelectKBest(chi2, k=k_best)
    X = ch2.fit_transform(X,labels)
    if print_feature_names:  # keep selected feature names.
        feature_names = vectorizer.get_feature_names()
        feature_names = [feature_names[i] for i in ch2.get_support(indices=True)]
        print(feature_names)
    

    print("--- %s seconds for feature computing ---" % (time.time() - start_time))
    return X, vectorizer, ch2





---



### 🔍 Fonction de recherche d'un classifieur optimisé 

Dans cette fonction nous allons utiliser un algorithme nommé Grid-Search (type *brute-force* 🥊) permettant d'optimiser les paramètres d'un ou plusieurs classifieur(s) puis d'en comparer les performances. Nous allons comparer 5 types de classifieurs disponibles dans Scikit Learn :
* Software Vector Machine (SVC ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html?highlight=svc#sklearn.svm.SVC))). Optimisation du paramètre <code>kernel</code> : `['linear','rbf']`
* LogisticRegression ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression)). Optimisation du paramètre <code>C</code> : `[0.5,0.75,1]`
* MultinomialNB ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html?highlight=multinomialnb#sklearn.naive_bayes.MultinomialNB)). Pas d'optimisation de paramètre. 
* RandomForestClassifier ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=randomforestclassifier#sklearn.ensemble.RandomForestClassifier)). Optimisation du paramètre <code>max_depth</code> : `[5,10,15]`
* RidgeClassifier ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html?highlight=ridgeclassifier#sklearn.linear_model.RidgeClassifier)) Optimisation du paramètre <code>alpha</code> : `[0.5,0.75,1]`

⚠ **Attention** Certains de ces classifieurs utilisent de l'aléatoire. Il est donc nécessaire de fixer les graines à l'aide du paramètre <code>random_state</code>

1. Nous allons utiliser un Pipeline dans lequel nous allons ajouter les dictionnaires contenant les classifieurs et leurs paramètres à ajuster. Pour cela vous allez vous pouvez vous inspirer du code suivant : 
```python
# Créé un Pipeline pseudo vide (workaround pour pouvoir utiliser le pipeline avec plusieurs classifieurs)
pipeline = Pipeline([
    ('clf', UnClassifieur()),
    ])
# Liste de dictionnaire pour chacun des classifieurs et leurs paramètres à tester
parameters = [
        {
            'clf': [UnClassifieur(random_state=0)],
            'clf__nomParamClassifier': ['linear','rbf']
        }
]
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
```

Dans le code plus haut le `pipeline` est initialisé avec un classifieur. C'est un workaround car on ne peut pas créer de `pipeline` vide. 
Nous allons utiliser ce `pipeline` avec une liste (variable <code>parameters</code>) de dictionnaires contenant le classifieur à tester et ses paramètres. 
Dans le dictionnaire présenté plus haut :
* <code>'clf'</code> est une liste contenant 1 seul élément, le classifieur à tester.
* <code>'clf__nomParamClassifier'</code> permet à l'algorithme de tester le paramètres <nomParamClassifier> du classifieur testé <code>'clf'</code> 

Pour ajouter les classifieurs à tester vous allez vous baser sur le meme principe, à savoir, rajouter des dictionnaires dans <code>parameters</code> avec 
<code>{'clf' : [mon_classif], 'clf__nomParamClassifier' : [liste_param_à_tester]}</code>. Vous prendrez la liste des classifieurs/paramètres donnée plus haut dans ce bloc. 

2. Vous devrez ensuite créer un objet GridSearchCV ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)) avec en paramètres : <code>pipeline</code>, <code>parameters</code> et <code>n_jobs=-1</code> (pour obtenir un multi-threading sur l'ensemble des coeurs CPU disponibles).
3. Vous utiliserez ensuite la méthode `fit(X_train,y_train)` sur l'objet GridSearch pour lancer la recherche du meilleur couple classifieurs/paramètres

In [ ]:
def grid_search_classifiers(X,binarized_labels):
    #Séparation du jeu de données en 2. Une partie pour l'entrainement (X_train, y_train) et une partie pour l'évaluation (X_test, y_test)
    X_train, X_test, y_train, y_test = train_test_split(X, binarized_labels, test_size=0.33, random_state=42)

    print("--- Start grid-search ---")
    start_time = time.time()
    
    #Placer ici le code avec le pipeline pour le Grid-Search
    pipeline = Pipeline([
    ('clf', SVC())])
    # Liste de dictionnaire pour chacun des classifieurs et leurs paramètres à tester
    parameters = [
        {
            'clf': [SVC(random_state=0)],
            'clf__kernel': ['linear','rbf']
        },
        {
            'clf': [LogisticRegression(random_state=0)],
            'clf__C': [0.5,0.75,1]
        },
        {
            'clf': [MultinomialNB()]
        },
        {
            'clf': [RandomForestClassifier(random_state=0)],
            'clf__max_depth': [5,10,15]
        },
        {
            'clf': [RidgeClassifier(random_state=0)],
            'clf__alpha': [0.5,0.75,1]
        }
    ]

    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
    grid_search.fit(X_train,y_train)

    print("Best parameters set found on development set:")
    print()
    print(grid_search.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = grid_search.cv_results_["mean_test_score"]
    stds = grid_search.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, grid_search.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, grid_search.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()
    
    print("--- %s seconds for grid-search ---" % (time.time() - start_time))
    return grid_search.best_estimator_


## ⏯ Création du bloc principal d'exécution du Grid Search
1. Chargement du jeu de données de Herzig et al. avec la méthode <code>load_dataset("dataset_herzig_etal.json")</code>
2. Multiplication du titre des tickets du dataset par un facteur 3 via la méthode <code>multiply_title</code>
3. Extraction du corpus et des labels via la méthode <code>get_corpus_labels</code>
3. Vectorisation à l'aide de TF-IDF. Pour cela, créez un objet Vectorizer à l'aide de <code>TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})</code>
4. Sélection des features représentatives grâce à la méthode <code>feature_computing</code>
5. Recherche d'un classifieur via la méthode GridSearch

In [ ]:
#Chargement du jeu de données JSON
dataset = load_dataset("dataset_herzig_etal.json")
#Multiplication du titre par un facteur 3 (ajustable)
multiply_title(dataset, 3)
#Extraction du corpus de tickets et des étiquettes 
corpus, labels = get_corpus_labels(dataset)
#Transformation des étiquettes textuelles en étiquettes binaires
binarized_labels = binarization_labels(labels)
#Création d'un vectorizer 
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})


#Extraction des features représentatives du corpus de tickets (ici 3000 features les plus réprésentatives)
X, vectorizer, ch2 = feature_computing(corpus, binarized_labels, vectorizer)

#Recherche du meilleur classifieur
grid_search_classifiers(X,binarized_labels)


1940 BUG / 5591 

--- Start feature computing ---
	99349 features.
Extracting 30000 best features by a chi-squared test
['00', '00 00', '00 00 00', '00 00 gmt', '00 gmt', '000 defaulthttpexecutor', '000 defaulthttpexecutor java', '00000001', '01', '01 00', '01 00 00', '01 gmt', '02', '02 01', '02 2007', '03', '03 04', '03 06', '03 15', '03 2008', '03 2010', '04', '05', '05 11', '05 jul', '06 gmt', '08', '086', '088', '089', '09', '09 03', '09 53', '090', '091', '0_24', '0_24 64', '0_24 64 bit', '0_25', '0_25 64', '0_25 64 bit', '0_26', '0_26 64', '0_26 64 bit', '0m1', '0rc2', '0rc3', '0x9', '10', '10 000000000', '10 09', '10 10', '10 17', '10 20', '10 but', '10 journal', '10 junit', '10 junit at', '10 junit junit', '10 param', '10 param name', '10 tid', '10 upgrade', '10 upgrade to', '100', '100 at', '100 at org', '100 continue', '100 for', '100 of', '100 string', '1005', '1005 junit', '1005 junit at', '1006', '1006 at', '1006 at org', '101', '101 at', '101 at org', '1010', '1010 at', 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Best parameters set found on development set:

{'clf': SVC(random_state=0), 'clf__kernel': 'rbf'}

Grid scores on development set:

0.869 (+/-0.025) for {'clf': SVC(random_state=0), 'clf__kernel': 'linear'}
0.879 (+/-0.016) for {'clf': SVC(random_state=0), 'clf__kernel': 'rbf'}
0.744 (+/-0.024) for {'clf': LogisticRegression(random_state=0), 'clf__C': 0.5}
0.773 (+/-0.019) for {'clf': LogisticRegression(random_state=0), 'clf__C': 0.75}
0.796 (+/-0.030) for {'clf': LogisticRegression(random_state=0), 'clf__C': 1}
0.732 (+/-0.016) for {'clf': MultinomialNB()}
0.681 (+/-0.017) for {'clf': RandomForestClassifier(random_state=0), 'clf__max_depth': 5}
0.699 (+/-0.017) for {'clf': RandomForestClassifier(random_state=0), 'clf__max_depth': 10}
0.713 (+/-0.024) for {'clf': RandomForestClassifier(random_state=0), 'clf__max_depth': 15}
0.875 (+/-0.023) for {'clf': RidgeClassifier(random_state=0), 'clf__alpha': 0.5}
0.870 (+/-0.023) for {'clf': RidgeClassifier(random_state=0), 'clf__alpha': 0.75}
0

Pipeline(steps=[('clf', SVC(random_state=0))])



---



## ✂ Échantillonage pour trouver un nombre de features optimal

Nous allons maintenant échantillonner le nombre de feature afin de trouver un nombre qui permet de donner de bons résultats.  
Pour cela nous allons faire varier le nombre de features entre 30000 et 60000.

1. Chargement du jeu de données de Herzig et al. avec la méthode <code>load_dataset("dataset_herzig_etal.json")</code>
2. Multiplication du titre des tickets du dataset par un facteur 3 via la méthode <code>multiply_title</code>
3. Extraction du corpus et des labels via la méthode <code>get_corpus_labels</code>
4. Vectorisation à l'aide de TF-IDF. Pour cela, créez un objet Vectorizer à l'aide de <code>TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})</code>
5. Sélection des features représentatives grâce à la méthode <code>feature_computing</code>
6. Création du classifieur, selectionner le meilleur couple classifieur/paramètres retourné par Grid-Search précédement
7. Échantillonage du nombre de features par pas de 5000 entre 30000 et 60000, fans la boucle faire un :
* calcul des features à l'aide de <code>feature_computing</code>
* le scoring à l'aide de la méthode <code>make_scoring_train_test_split</code>

En cas d'égalisté des scores F1 entre différents nombre de features, vous conserverez la valeur du plus grand nombre de features parmis les meilleurs scores.

In [ ]:
#Chargement du jeu de données JSON
dataset =load_dataset("dataset_herzig_etal.json")
#Multiplication du titre par un facteur 3 (ajustable)
multiply_title(dataset, 3)
#Extraction du corpus de tickets et des étiquettes 
corpus, labels = get_corpus_labels(dataset)
#Transformation des étiquettes textuelles en étiquettes binaires
binarized_labels = binarization_labels(labels)
#Création d'un vectorizer Tf-Idf
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})

#Extraction des features représentatives du corpus de tickets (ici 30000 features les plus réprésentatives)
X, vectorizer, ch2 = feature_computing(corpus, binarized_labels, vectorizer)


#Création du classifieur
clf = grid_search_classifiers(X,binarized_labels)

#Boucle for avec calcul des features et scoring
f1_best = 0
nb_feature = 0
for k_best in range(30000, 60001, 5000):
  X, vectorizer, ch2 = feature_computing(corpus, binarized_labels, vectorizer,k_best)
  f1 = make_scoring_train_test_split(X,binarized_labels, clf)
  if(f1_best <= f1):
    f1_best = f1
    nb_feature = k_best

  print("Meilleur score : %0.3f" % f1_best )
  print(f"nombre de features : \t{nb_feature}.")






## 🔢 Création de la matrice de confusion


Nous allons maintenant créer la matrice de confusion correspondant à notre classifieur. Cette matrice permet de visualiser graphiquement la qualité de la classification effectuée par notre classifieur. La matrice de confusion recense le nombre de :
* vrais positifs (VP)
* vrais négatifs (VN)
* faux positifs (FP)
* faux négatifs (FN)
Cette matrice est un indicateur de la qualité de votre classifieur. Plus le nombre de FP et FN est réduit meilleure est la classification. 

Pour créer cette matrice il vous faut : 
1. Comme dans les blocs de code précédent charger le jeu, multiplier le titre, extraire le corpus et les labels puis binariser les labels
2. Extraire les <code>k</code> meilleures features en fonction de l'échantillonage fait plus haut (méthode <code>feature_computing</code>)
3. Utiliser la meilleure configuration de classifieur calculée avec Grid-Search 
4. Entraîner ce classifieur à l'aide de la méthode <code>fit</code> avec <code>X_train</code> et <code>y_train</code>
5. Faire des prédictions à l'aide de la méthode <code>predict</code> de votre classifieur

Pour cela, vous pouvez vous inspirer de l'exemple donné ici : [https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html)

In [ ]:
#Chargement du jeu de données JSON
dataset =load_dataset("dataset_herzig_etal.json")
#Multiplication du titre par un facteur 3 (ajustable)
multiply_title(dataset, 3)
#Extraction du corpus de tickets et des étiquettes 
corpus, labels = get_corpus_labels(dataset)
#Transformation des étiquettes textuelles en étiquettes binaires
binarized_labels = binarization_labels(labels)
#Création du vectorizer TF-IDF et sélection des k-best features avec feature_computing
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})
X, vectorizer, ch2 = feature_computing(corpus, binarized_labels, vectorizer)

X_train, X_test, y_train, y_test = train_test_split(X, binarized_labels, test_size=0.33, random_state=42)

#Création du classifieur, entrainement avec X_train, y_train et prédiction avec X_test
# clf = grid_search_classifiers(X,binarized_labels)
clf = SVC(random_state = 0, kernel='rbf', probability=True)

clf.fit(X_train,y_train)

y_true, y_pred = y_test, clf.predict(X_test)


#affichage de la matrice de confusion ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=clf.classes_)
disp.plot()

plt.show()



---



## ❓ Explicabilité du classifieur

Un des défis lié à l'IA et aux algorithmes à trait à leur explicabilité. L'explicabilité se définie comme le fait de pouvoir comprendre les mécanismes internes du classifieur qui fondent une ou plusieurs prédictions. Cette explicabilité peut se faire de manière globale (mécanismes internes du classifieur qui conduisent à la classification) ou de manière locale (mécanismes qui conduisent à la classification d'une instance). 

Nous allons expliquer la classification de 4 tickets : 
* 2 tickets sont des faux positifs (indices 3997 et 5098 dans le dataset de tickets)
* 2 tickets sont des faux négatifs (indices 2656 et 3479 dans le dataset de tickets)

Pour expliquer les mots impactant la classification des tickets nous allons utiliser une méthode d'explication se nommant Lime située dans le package Python [eli5](https://eli5.readthedocs.io/en/latest/index.html). 
Pour cela vous pouvez vous inspirer de l'exemple donner dans la documentation de eli5 : [https://eli5.readthedocs.io/en/latest/tutorials/black-box-text-classifiers.html#textexplainer](https://eli5.readthedocs.io/en/latest/tutorials/black-box-text-classifiers.html#textexplainer)

**Attention** avant d'exécuter le code suivant, veillez à avoir exécuté le code du bloc précédent (code de matrice de confusion)

In [ ]:
#Pipeline et vectorizer nécessaire pour le TextExplainer
pipe = make_pipeline(vectorizer, ch2, clf)
vectorizer_for_text_explainer = TfidfVectorizer(min_df=1, max_df=1.0,ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})

#Placez ici le code du text explainer pour le ticket 5098 (ticket_dataset[5098]["title"] + ticket_dataset[5098]["body"]
te = TextExplainer(random_state=42,vec=vectorizer_for_text_explainer)
te.fit(dataset[5098]["title"] + dataset[5098]["body"], pipe.predict_proba)
te.show_prediction(target_names=["NBUG","BUG"])

Placez ci-dessous le code du text explainer pour le ticket 3997 (faux positif)

In [ ]:
#Pipeline et vectorizer nécessaire pour le TextExplainer
pipe = make_pipeline(vectorizer, ch2, clf)
vectorizer_for_text_explainer = TfidfVectorizer(min_df=1, max_df=1.0,ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})

#Placez ici le code du text explainer pour le ticket 3997 (ticket_dataset[3997]["title"] + ticket_dataset[3997]["body"]
te = TextExplainer(random_state=42,vec=vectorizer_for_text_explainer)
te.fit(dataset[3997]["title"] + dataset[3997]["body"], pipe.predict_proba)
te.show_prediction(target_names=["NBUG","BUG"])

Placez ci-dessous le code du text explainer pour le ticket 2656 (faux négatif)

In [ ]:
#Pipeline et vectorizer nécessaire pour le TextExplainer
pipe = make_pipeline(vectorizer, ch2, clf)
vectorizer_for_text_explainer = TfidfVectorizer(min_df=1, max_df=1.0,ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})

#Placez ici le code du text explainer pour le ticket 2656 (ticket_dataset[2656]["title"] + ticket_dataset[2656]["body"]
te = TextExplainer(random_state=42,vec=vectorizer_for_text_explainer)
te.fit(dataset[2656]["title"] + dataset[2656]["body"], pipe.predict_proba)
te.show_prediction(target_names=["NBUG","BUG"])

Placez ci-dessous le code du text explainer pour le ticket 3479 (faux négatif)

In [ ]:
#Pipeline et vectorizer nécessaire pour le TextExplainer
pipe = make_pipeline(vectorizer, ch2, clf)
vectorizer_for_text_explainer = TfidfVectorizer(min_df=1, max_df=1.0,ngram_range=(1, 3), sublinear_tf=True, stop_words={'english'})

#Placez ici le code du text explainer pour le ticket 3479 (ticket_dataset[3479]["title"] + ticket_dataset[3479]["body"]
te = TextExplainer(random_state=42,vec=vectorizer_for_text_explainer)
te.fit(dataset[3479]["title"] + dataset[3479]["body"], pipe.predict_proba)
te.show_prediction(target_names=["NBUG","BUG"])